## Accleration from motion
Calcualtion of 6 DOF acceleration in some point based on 6 DOF motion in some other point

In [2]:
import warnings
#warnings.filterwarnings('ignore')
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [3]:
import sympy as sp
import sympy.physics.mechanics as me
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from signal_lab.substitute_dynamic_symbols import substitute_dynamic_symbols, find_name, find_derivative_name, lambdify, find_derivatives


ImportError: No module named 'signal_lab'

In [ ]:
x0,y0,z0 = me.dynamicsymbols('x0 y0 z0')
x01d,y01d,z01d = me.dynamicsymbols('x0 y0 z0',1)
x02d,y02d,z02d = me.dynamicsymbols('x0 y0 z0',2)

phi,theta,psi = me.dynamicsymbols('phi theta psi')
p,q,r = me.dynamicsymbols('phi theta psi',1)
p1d,q1d,r1d = me.dynamicsymbols('phi theta psi',2)


p_x = sp.Symbol('p_x')
p_y = sp.Symbol('p_y')
p_z = sp.Symbol('p_z')



In [ ]:
x02d

In [ ]:
N = me.ReferenceFrame('N')

In [ ]:
#S = N.orientnew('S', 'Body', [phi,theta,psi],'321')
#M = N.orientnew('M','Axis', [psi,N.z])

S = N.orientnew('S', 'Body', [psi,theta,phi],'ZYX')


In [ ]:
#S.set_ang_vel(N, q * N.y)
#S.set_ang_acc(N, q1d * N.y)

In [ ]:
S.ang_vel_in(N)

In [ ]:
S.ang_acc_in(N)

In [ ]:
M = me.Point('M')  # Origo of measurement system
M.set_vel(N, 0)

O = M.locatenew('O', x0*N.x + y0*N.y + z0*N.z) # Origo of ship
O.set_vel(N, x01d*N.x + y01d*N.y + z01d*N.z)
O.set_acc(N, x02d*N.x + y02d*N.y + z02d*N.z)



In [ ]:
P = O.locatenew('P', p_x*S.x + p_y*S.y + p_z*S.z)
P.set_vel(S, 0)


In [ ]:
S.dcm(N)

In [ ]:
vector = P.pos_from(M)
motion_P = vector.express(N)
motion_P

In [ ]:
motion_matrix = vector.to_matrix(N)
motion_matrix

In [ ]:
motion_lambda = lambdify(motion_matrix)

In [ ]:
motion_lambda

In [ ]:
number_of_points = 100
t = np.linspace(0,2*np.pi,number_of_points)
df = pd.DataFrame(index = t)
df['phi'] = 0
df['theta'] = np.sin(t)
df['psi'] = 0

df['p_x'] = 20
df['p_y'] = 0
df['p_z'] = 1

df['x0'] = np.sin(t)
df['y0'] = np.sin(t)
df['z0'] = -np.sin(t)
motions = motion_lambda(**df)

df_result = df.copy()
df_result['x'] = motions[0][0]
df_result['y'] = motions[1][0]
df_result['z'] = motions[2][0]

fig,ax=plt.subplots()
df_result.plot(y='x', ax=ax)

fig,ax=plt.subplots()
df_result.plot(y='z', ax=ax)


## Velocity

In [ ]:
P.v2pt_theory(O,N,S)

In [ ]:
velocity_matrix = P.vel(N).express(N).to_matrix(N)
velocity_matrix

In [ ]:
velocity_lambda = lambdify(velocity_matrix)

In [ ]:
velocity_lambda

In [ ]:
number_of_points = 100
t = np.linspace(0,2*np.pi,number_of_points)
df = pd.DataFrame(index = t)
df['phi'] = 0
df['theta'] = np.sin(t)
df['psi'] = 0

df['phi1d'] = 0
df['theta1d'] = np.cos(t)
df['psi1d'] = 0

df['p_x'] = 20
df['p_y'] = 0
df['p_z'] = 1

df['x01d'] = 10
df['y01d'] = 0
df['z01d'] = 0


velocities = velocity_lambda(**df)

df_result = df.copy()
df_result['x01d'] = velocities[0][0]
df_result['y01d'] = velocities[1][0]
df_result['z01d'] = velocities[2][0]

fig,ax=plt.subplots()
df_result.plot(y='x01d', ax=ax)

fig,ax=plt.subplots()
df_result.plot(y='z01d', ax=ax)

## Acceleration

In [ ]:
P.a2pt_theory(O,N,S)

### Acceleration in earth fixed reference frame N expressed in ship Reference frame S

In [ ]:
#acceleration_matrix = P.acc(N).express(N).to_matrix(N)
acceleration_matrix = P.acc(N).express(S).to_matrix(S)

acceleration_matrix

In [ ]:
substitute_dynamic_symbols(acceleration_matrix)

In [ ]:
acceleration_lambda = lambdify(acceleration_matrix)

In [ ]:
acceleration_lambda

### Centrifugal acceleration
Absolute acceleration can be calcualted as: <br>
$ |a| = r\cdot\dot{\theta}^2 $

With the components: <br>
$ \ddot{x_0} = -a $ <br>
$ \ddot{z_0} = 0 $


In [ ]:
number_of_points = 100
t = np.linspace(0,2*np.pi,number_of_points)
r_ = 10
w_ = 1
df = pd.DataFrame(index = t)

df['phi'] = 0
df['theta'] = w_*t
df['psi'] = 0

df['phi1d'] = 0
df['theta1d'] = w_
df['psi1d'] = 0

df['phi2d'] = 0
df['theta2d'] = 0
df['psi2d'] = 0

df['p_x'] = r_
df['p_y'] = 0
df['p_z'] = 0


df['x02d'] = 0
df['y02d'] = 0
df['z02d'] = 0


accelerations = acceleration_lambda(**df)

df_result = df.copy()
df_result['x02d'] = accelerations[0,0]
df_result['y02d'] = accelerations[1,0]
df_result['z02d'] = accelerations[2,0]

a = r_* w_**2
df_result['x02d_check'] = -a
df_result['z02d_check'] = 0

fig,ax=plt.subplots()
df_result.plot(y=['x02d','x02d_check'], ax=ax, alpha=0.7)

fig,ax=plt.subplots()
df_result.plot(y=['z02d','z02d_check'], ax=ax, alpha=0.7)

## Save lambdas

In [ ]:
import dill
import signal_lab
dill.settings['recurse'] = True

In [ ]:
import os.path
with open(os.path.join(signal_lab.directory,'point_acceleration.dill'),mode='wb') as file:
    dill.dump(acceleration_lambda, file=file)